In [1]:
import pandas as pd
from error_analysis import main
import os
from os import listdir
from os.path import isfile, join
from IPython.display import display, HTML

In [2]:
gold_dir = "/home/lisa/projects/cross_ling_drug_ner/data/converted_fixed_v2/test/"
predicted_dir = "/home/lisa/projects/cross_ling_drug_ner/models_ensembled/fixed_data/all_on_all/unified/ensembled_predictions/"
results_dir = "/".join(predicted_dir.split("/")[:-2])
print(f"results dir: {results_dir}")

results dir: /home/lisa/projects/cross_ling_drug_ner/models_ensembled/fixed_data/all_on_all/unified


In [3]:
file_prefixes = set(["_".join(f.split("_")[:2]) for f in listdir(gold_dir) if isfile(join(gold_dir, f))])
file_prefixes

{'de_bronco150',
 'de_ex4cds',
 'de_gernermed',
 'de_gponc',
 'en_cmed',
 'es_ct',
 'es_pharmaconer',
 'fr_deft',
 'fr_quaero'}

In [4]:
language = "es"
# language can be: ["all", "de", "en", "es", "fr"] but also the beginning of the file names per corpus,
# to get corpus specific results: "de_gponc", "de_gernermed", etc.
df_strict, df_lenient = main(gold_dir, predicted_dir, verbose=False, language=language)


******************** Evaluation n2c2 2022 Track 1 ********************
************* Contextualized Medication Event Extraction *************

*********************** Medication Extraction ************************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)
                Drug	0.8239	0.8789	0.8505	0.8565	0.9136	0.8841


************************ Event Classification ************************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)


In [5]:

def highlight_selected_text(entity, text):
    return text.replace(entity, f'<span style="color: green; font-weight: bold">{entity}</span>')

def get_unique_entites(false_entities, file_names):
    unique = set()
    errors_per_file = []
    for file_name, l in zip(file_names, false_entities):
        if not l:
            continue
        with open(os.path.join(gold_dir, file_name.replace(".ann", ".txt")), "r") as read_handle:
            text = read_handle.read()
        for element in l:
            unique.add(element[0])
            errors_per_file.append({"file": file_name, 
                                    "entity": element[0], 
                                    "start": element[1], 
                                    "end": element[2],
                                    "sentence_h": highlight_selected_text(entity=element[0],
                                                                          text="..." + text[element[1] - 200: \
                                                                                          element[2] + 200] + "..."),
                                    "sentence": "..." + text[element[1] - 200: element[2] + 200] + "..." 
                                   })
    
    return unique, pd.DataFrame(errors_per_file).sort_values("file").reset_index(drop=True)

def clean_up_dfs(df):
    # sort the dataframe by file name
    df = df.sort_values("file").reset_index(drop=True)
    
    # convert FP and FN columns to list and then to their own DFs
    fps = df['FP'].tolist()
    unique_fps, fps_per_file = get_unique_entites(fps, list(df["file"]))
    fns = df['FN'].tolist()
    unique_fns, fns_per_file = get_unique_entites(fns, list(df["file"]))
    
    df_fp = pd.DataFrame(fps, index=df.index).add_prefix('FP')
    df_fn = pd.DataFrame(fns, index=df.index).add_prefix('FN')
    
    # remove the old FP and FN columns and concatenate the three DFs
    df = pd.concat([df.drop(columns=['FP', "FN"]), df_fp, df_fn], axis=1)
    
    df = df.round(2)
 
    return df, unique_fps, unique_fns, fps_per_file, fns_per_file

In [6]:
df_strict, unique_fps_strict, unique_fns_strict, fps_per_file_strict, fns_per_file_strict = clean_up_dfs(df_strict)
df_lenient, unique_fps_lenient, unique_fns_lenient, fps_per_file_lenient, fns_per_file_lenient = clean_up_dfs(df_lenient)

In [7]:
print(f"max #false negatives: {df_strict['#FN'].max()}")
print(f"max #false positives: {df_strict['#FP'].max()}")
print(f"max #gold entities: {df_strict['#gold_entities'].max()}")

max #false negatives: 15
max #false positives: 16
max #gold entities: 41


In [8]:
fns_per_file_lenient_csv = fns_per_file_lenient.drop(columns="sentence_h")
fns_per_file_lenient_html = fns_per_file_lenient.drop(columns="sentence")

fns_per_file_lenient_html.style.set_sticky(axis="index")
html_table = fns_per_file_lenient_html.to_html(escape=False)
display(HTML(html_table))

,file,entity,start,end,sentence_h
0,es_ct_ebm_sp_0211-699500012506.ann,alumínica,827,836,"...ro objetivo fue valorar el efecto de un concentrado de 2,5 mEq/l de calcio sobre la masa ósea en los pacientes con PTH suprimida.\nEstudiamos 20 pacientes con PTH intacta < 120 pg/ml, sin intoxicación alumínica, trasplante o paratiroidectomía, en hemodiálisis con 3 mEq/l de calcio y CO3Ca como captor del fósforo.\nSexo: 10 hombres/10 mujeres, edad: 57 ± 13 años, meses en diálisis: 40 ± 29.\nSe valoró la masa ..."
1,es_ct_ebm_sp_0211-699500012506.ann,PTH,742,745,"...nte a 2,5 mEq/l.\nSe desconoce la repercusión de esta medida sobre la masa ósea.\nNuestro objetivo fue valorar el efecto de un concentrado de 2,5 mEq/l de calcio sobre la masa ósea en los pacientes con PTH suprimida.\nEstudiamos 20 pacientes con PTH intacta < 120 pg/ml, sin intoxicación alumínica, trasplante o paratiroidectomía, en hemodiálisis con 3 mEq/l de calcio y CO3Ca como captor del fósforo.\nSexo..."
2,es_ct_ebm_sp_0211-699502014623.ann,hidrocarbonado,2333,2347,"...abéticos tipo 2 con albuminuria estable, ambos tratamientos reducen la presión arterial y la albuminuria de forma similar.\nLa combinación verapamil/trandolapril contribuye mejor al control metabólico hidrocarbonado que losartan/hidroclorotiazida.\n\n..."
3,es_ct_ebm_sp_2011-006287-50.ann,Extracto,251,259,...o Clínico del Extracto de membrana amniótica como tratamiento del ojo seco severo en comparación con el tratamiento mdiante colirio de suero autólogo \n\nTítulo científico: Ensayo Clínico fase III del Extracto de membrana amniótica como tratamiento sintomático del ojo seco severo en comparación con el tratamiento mdiante colirio de suero autólogo \n\nIndicación pública: Ojo seco severo \n\nCriterios de inclus...
4,es_ct_ebm_sp_2012-001610-42.ann,midriáticos,1567,1578,...arada \n- Uveítis anterior aislada\n- Agudeza visual mejorada < 35 letras en el ojo no estudiado o visión monocular \n- Presencia de cataratas u opacificación capsular grave\n- Contraindicación de uso de midriáticos o presencia de sinequias posteriores.\n- Enfermedad tuberculosis activa. \n- Antecedentes de reacción alérgica o anafiláctica grave a anticuerpos monoclonales \n- Antecedentes de neoplasia en lops 5 año...
5,es_ct_ebm_sp_2012-001984-66.ann,proteína C,1034,1044,"...ón y con dosis estables durante al menos las 8 semanas anteriores a dicha visita\nEnfermedad activa, definida como: al menos 6 de 66 articulaciones hinchadas y 8 de 68 articulaciones hipersensibles, y proteína C reactiva de alta sensibilidad (PCR-as) superior a 10 mg/l \n\nCriterios de exclusión:\nEdad inferior a 18 años\nAdministración de corticoesteroides por vía parenteral o intraarticular en las 4 semanas an..."
6,es_ct_ebm_sp_2013-001229-15.ann,contraste,636,645,"...ica con contraste \n\nIndicación científica: Pacientes con alto riesgo de desarrollar nefropatía por contraste \n\nCriterios de inclusión:\nPacientes que vayan a ser sometidos a la realización de TAC con contraste, según indicación de práctica clínica, y que tras haber recibir información sobre el diseño, los fines del estudio, los posibles riesgos que de él pueden derivarse, y denegación de continuar en el es..."
7,es_ct_ebm_sp_2013-001526-26.ann,MK 0822,286,293,"...protocolo 018 \n\nTítulo científico: Estudio de seguimiento observacional de un ensayo clínico aleatorizado, controlado con placebo, en fase III, para evaluar la seguridad y la eficacia de odanacatib (MK 0822) en cuanto a la reducción del riesgo de fractura en mujeres posmenopáusicas con osteoporosis tratadas con vitamina D y calcio (protocolo 018) \n\nIndicación pública: Osteoporosis \n\nCriterios de inclus..."
8,es_ct_ebm_sp_2013-001835-33.ann,LCZ696,79,85,......
9,es_ct_ebm_sp_2013-001835-33.ann,Potasio,694,701,"...II-IV de la NYHA.\n- FEVI ≤ 35% en la selección.\n- Los pacientes deben ser tratados con un β bloqueante\nVer resto de criterios del protocolo \n\nCriterios de exclusión:\nTFG estimada < 30 ml/min/1,73 m2 \nPotasio sérico

In [9]:
fps_per_file_lenient_csv = fps_per_file_lenient.drop(columns="sentence_h")
fps_per_file_lenient_html = fps_per_file_lenient.drop(columns="sentence")

fps_per_file_lenient_html.style.set_sticky(axis="index")
html_table = fps_per_file_lenient_html.to_html(escape=False)
display(HTML(html_table))

,file,entity,start,end,sentence_h
0,es_ct_ebm_sp_0211-699500012506.ann,CO3Ca,2357,2362,"...± 38, p < 0,01), sin cambios en grupo II.\nLa pérdida ósea se relacionó inversamente con incremento de PTH (r = -0,61, p < 0,01).\nCalcio y fósforo no se modificaron.\nEn grupo I la dosis acumulativa de CO3Ca se incrementó: 1.er trimestre: 332 ± 261; 4.º trimestre: 537 ± 260 (expresado como calcio elemento; p < 0,01).\nEn análisis de regresión lineal múltiple, un mayor incremento de PTH y el tipo de concen..."
1,es_ct_ebm_sp_0211-699500012506.ann,calcio,1396,1402,"...lares en sexo, edad y tiempo en diálisis.\nSin diferencias en densidad ósea, PTH, calcio, fósforo y fosfatasa alcalina.\nEl grupo I fue transferido a 2,5 mEq/l y el II continuó con 3 mEq/l.\nSe midieron calcio, fósforo (mg/dl) y fosfatasa alcalina (UI/l) mensualmente.\nPTH (pg/ml) cada tres meses y la densidad ósea (mg/cc) al término del estudio.\nTras un año de hemodiálisis con 2,5 mEq/l de calcio hubo una ..."
2,es_ct_ebm_sp_0211-699500012506.ann,captor,921,927,"...en los pacientes con PTH suprimida.\nEstudiamos 20 pacientes con PTH intacta < 120 pg/ml, sin intoxicación alumínica, trasplante o paratiroidectomía, en hemodiálisis con 3 mEq/l de calcio y CO3Ca como captor del fósforo.\nSexo: 10 hombres/10 mujeres, edad: 57 ± 13 años, meses en diálisis: 40 ± 29.\nSe valoró la masa ósea trabecular en columna vertebral mediante tomografía axial computerizada cuantitativa.\n..."
3,es_ct_ebm_sp_0211-699500012506.ann,fósforo,1404,1411,"... sexo, edad y tiempo en diálisis.\nSin diferencias en densidad ósea, PTH, calcio, fósforo y fosfatasa alcalina.\nEl grupo I fue transferido a 2,5 mEq/l y el II continuó con 3 mEq/l.\nSe midieron calcio, fósforo (mg/dl) y fosfatasa alcalina (UI/l) mensualmente.\nPTH (pg/ml) cada tres meses y la densidad ósea (mg/cc) al término del estudio.\nTras un año de hemodiálisis con 2,5 mEq/l de calcio hubo una reducción..."
4,es_ct_ebm_sp_0211-699500012506.ann,concentrado,2556,2567,"... CO3Ca se incrementó: 1.er trimestre: 332 ± 261; 4.º trimestre: 537 ± 260 (expresado como calcio elemento; p < 0,01).\nEn análisis de regresión lineal múltiple, un mayor incremento de PTH y el tipo de concentrado utilizado se comportaron como únicos predictores de pérdida ósea (r = 0,74, p < 0,01).\nEn conclusión, el uso de un concentrado de 2,5 mEq/l de calcio resultó en: 1.º Pérdida de masa ósea. 2.º Aumento..."
5,es_ct_ebm_sp_0211-699500012506.ann,calcio,1277,1283,"...ante tomografía axial computerizada cuantitativa.\nSe dividieron en grupo I (11 pacientes) y grupo II (9 pacientes) similares en sexo, edad y tiempo en diálisis.\nSin diferencias en densidad ósea, PTH, calcio, fósforo y fosfatasa alcalina.\nEl grupo I fue transferido a 2,5 mEq/l y el II continuó con 3 mEq/l.\nSe midieron calcio, fósforo (mg/dl) y fosfatasa alcalina (UI/l) mensualmente.\nPTH (pg/ml) cada tres..."
6,es_ct_ebm_sp_0211-699500012506.ann,Calcio,2286,2292,"...99 ± 69, p < 0,01) y fosfatasa alcalina (basal: 118,4 ± 47, final: 152 ± 38, p < 0,01), sin cambios en grupo II.\nLa pérdida ósea se relacionó inversamente con incremento de PTH (r = -0,61, p < 0,01).\nCalcio y fósforo no se modificaron.\nEn grupo I la dosis acumulativa de CO3Ca se incrementó: 1.er trimestre: 332 ± 261; 4.º trimestre: 537 ± 260 (expresado como calcio elemento; p < 0,01).\nEn análisis de reg..."
7,es_ct_ebm_sp_0211-699500012506.ann,fósforo,1285,1292,"...ografía axial computerizada cuantitativa.\nSe dividieron en grupo I (11 pacientes) y grupo II (9 pacientes) similares en sexo, edad y tiempo en diálisis.\nSin diferencias en densidad ósea, PTH, calcio, fósforo y fosfatasa alcalina.\nEl grupo I fue transferido a 2,5 mEq/l y el II continuó con 3 mEq/l.\nSe midieron calcio, fósforo (mg/dl) y fosfatasa alcalina (UI/l) mensualmente.\nPTH (pg/ml) cada tres meses y ..."
8,es_ct_ebm_sp_0211-699500012506.ann,fósforo,2295,2302,"...p < 0,01) y fosfatasa alcalina (basal: 118,4 ± 47, final: 152 ± 38, p < 0,01), sin cambios en grupo II.\nLa pér

In [10]:
# FN = entities that were not detected by sys
fns_per_file_strict_csv = fns_per_file_strict.drop(columns="sentence_h")
#.to_csv(os.path.join(results_dir, "fns_strict.csv"))
fns_per_file_strict_html = fns_per_file_strict.drop(columns="sentence")

fns_per_file_strict_html.style.set_sticky(axis="index")
html_table = fns_per_file_strict_html.to_html(escape=False)
display(HTML(html_table))

,file,entity,start,end,sentence_h
0,es_ct_ebm_sp_0211-699500012506.ann,calcio elemento,2446,2461,"...ncremento de PTH (r = -0,61, p < 0,01).\nCalcio y fósforo no se modificaron.\nEn grupo I la dosis acumulativa de CO3Ca se incrementó: 1.er trimestre: 332 ± 261; 4.º trimestre: 537 ± 260 (expresado como calcio elemento; p < 0,01).\nEn análisis de regresión lineal múltiple, un mayor incremento de PTH y el tipo de concentrado utilizado se comportaron como únicos predictores de pérdida ósea (r = 0,74, p < 0,01).\nEn con..."
1,es_ct_ebm_sp_0211-699500012506.ann,alumínica,827,836,"...ro objetivo fue valorar el efecto de un concentrado de 2,5 mEq/l de calcio sobre la masa ósea en los pacientes con PTH suprimida.\nEstudiamos 20 pacientes con PTH intacta < 120 pg/ml, sin intoxicación alumínica, trasplante o paratiroidectomía, en hemodiálisis con 3 mEq/l de calcio y CO3Ca como captor del fósforo.\nSexo: 10 hombres/10 mujeres, edad: 57 ± 13 años, meses en diálisis: 40 ± 29.\nSe valoró la masa ..."
2,es_ct_ebm_sp_0211-699500012506.ann,PTH,742,745,"...nte a 2,5 mEq/l.\nSe desconoce la repercusión de esta medida sobre la masa ósea.\nNuestro objetivo fue valorar el efecto de un concentrado de 2,5 mEq/l de calcio sobre la masa ósea en los pacientes con PTH suprimida.\nEstudiamos 20 pacientes con PTH intacta < 120 pg/ml, sin intoxicación alumínica, trasplante o paratiroidectomía, en hemodiálisis con 3 mEq/l de calcio y CO3Ca como captor del fósforo.\nSexo..."
3,es_ct_ebm_sp_0211-699500012506.ann,concentrado para diálisis,458,483,"...dictivo de enfermedad ósea adinámica no relacionada con el aluminio.\nEste hipoparatiroidismo relativo se ve favorecido por los balances positivos de calcio producidos por los compuestos cálcicos y el concentrado para diálisis habiéndose propuesto la reducción del calcio en el dializante a 2,5 mEq/l.\nSe desconoce la repercusión de esta medida sobre la masa ósea.\nNuestro objetivo fue valorar el efecto de un concentrado de 2..."
4,es_ct_ebm_sp_0211-699500012698.ann,calcio elemento,858,873,"...upo 1: 14 pacientes que reciben carbonato cálcico a dosis de 2.500 mg/día (1.000 mg de calcio elemento); Grupo 2: 14 pacientes que reciben acetato cálcico a dosis de 1.000 mg (equivalente a 254 mg de calcio elemento). Ambos grupos realizaron dieta baja fósforo. El período de seguimiento fue de 24 meses. Se hicieron determinaciones analíticas de calcio y fósforo trimestral y PTH, F, alcalina y ClCr semestralmente..."
5,es_ct_ebm_sp_0211-699500012698.ann,calcio elemento,745,760,...lisis.\n\nMaterial y métodos: Estudiamos 28 pacientes con IRC avanzada (CICr 21 ml/min) divididos en dos grupos: Grupo 1: 14 pacientes que reciben carbonato cálcico a dosis de 2.500 mg/día (1.000 mg de calcio elemento); Grupo 2: 14 pacientes que reciben acetato cálcico a dosis de 1.000 mg (equivalente a 254 mg de calcio elemento). Ambos grupos realizaron dieta baja fósforo. El período de seguimiento fue de 24 mese...
6,es_ct_ebm_sp_0211-699502014623.ann,hidrocarbonado,2333,2347,"...abéticos tipo 2 con albuminuria estable, ambos tratamientos reducen la presión arterial y la albuminuria de forma similar.\nLa combinación verapamil/trandolapril contribuye mejor al control metabólico hidrocarbonado que losartan/hidroclorotiazida.\n\n..."
7,es_ct_ebm_sp_2011-004695-11.ann,Peginterferón Lambda-1a,91,114,......
8,es_ct_ebm_sp_2011-006287-50.ann,Extracto,251,259,...o Clínico del Extracto de membrana amniótica como tratamiento del ojo seco severo en comparación con el tratamiento mdiante colirio de suero autólogo \n\nTítulo científico: Ensayo Clínico fase III del Extracto de membrana amniótica como tratamiento sintomático del ojo seco severo en comparación con el tratamiento mdiante colirio de suero autólogo \n\nIndicación pública: Ojo seco severo \n\nCriterios de inclus...
9,es_ct_ebm_sp_2012-000797-35.ann,ANÁLOGO DE PURINAS,377,395,...ALFA-2A PEGILADO EN SUJETOS CON MUTACIÓN D816V DE KIT (U OTRAS MUTACIONES EN EL EXÓN 17 DEL KIT). \n\nTítulo científico: ENSAYO 

In [11]:
# FP = entities that were detected by sys but are not gold

fps_per_file_strict_csv = fps_per_file_strict.drop(columns="sentence_h")
#.to_csv(os.path.join(results_dir,"fps_strict.csv"))
fps_per_file_strict_html = fps_per_file_strict.drop(columns="sentence")

fps_per_file_strict_html.style.set_sticky(axis="index")
html_table = fps_per_file_strict_html.to_html(escape=False)
display(HTML(html_table))

,file,entity,start,end,sentence_h
0,es_ct_ebm_sp_0211-699500012506.ann,CO3Ca,2357,2362,"...± 38, p < 0,01), sin cambios en grupo II.\nLa pérdida ósea se relacionó inversamente con incremento de PTH (r = -0,61, p < 0,01).\nCalcio y fósforo no se modificaron.\nEn grupo I la dosis acumulativa de CO3Ca se incrementó: 1.er trimestre: 332 ± 261; 4.º trimestre: 537 ± 260 (expresado como calcio elemento; p < 0,01).\nEn análisis de regresión lineal múltiple, un mayor incremento de PTH y el tipo de concen..."
1,es_ct_ebm_sp_0211-699500012506.ann,fosfatasa alcalina,1295,1313,"...ial computerizada cuantitativa.\nSe dividieron en grupo I (11 pacientes) y grupo II (9 pacientes) similares en sexo, edad y tiempo en diálisis.\nSin diferencias en densidad ósea, PTH, calcio, fósforo y fosfatasa alcalina.\nEl grupo I fue transferido a 2,5 mEq/l y el II continuó con 3 mEq/l.\nSe midieron calcio, fósforo (mg/dl) y fosfatasa alcalina (UI/l) mensualmente.\nPTH (pg/ml) cada tres meses y la densidad ósea (mg/..."
2,es_ct_ebm_sp_0211-699500012506.ann,fósforo,2295,2302,"...p < 0,01) y fosfatasa alcalina (basal: 118,4 ± 47, final: 152 ± 38, p < 0,01), sin cambios en grupo II.\nLa pérdida ósea se relacionó inversamente con incremento de PTH (r = -0,61, p < 0,01).\nCalcio y fósforo no se modificaron.\nEn grupo I la dosis acumulativa de CO3Ca se incrementó: 1.er trimestre: 332 ± 261; 4.º trimestre: 537 ± 260 (expresado como calcio elemento; p < 0,01).\nEn análisis de regresión lin..."
3,es_ct_ebm_sp_0211-699500012506.ann,fósforo,1285,1292,"...ografía axial computerizada cuantitativa.\nSe dividieron en grupo I (11 pacientes) y grupo II (9 pacientes) similares en sexo, edad y tiempo en diálisis.\nSin diferencias en densidad ósea, PTH, calcio, fósforo y fosfatasa alcalina.\nEl grupo I fue transferido a 2,5 mEq/l y el II continuó con 3 mEq/l.\nSe midieron calcio, fósforo (mg/dl) y fosfatasa alcalina (UI/l) mensualmente.\nPTH (pg/ml) cada tres meses y ..."
4,es_ct_ebm_sp_0211-699500012506.ann,calcio,1277,1283,"...ante tomografía axial computerizada cuantitativa.\nSe dividieron en grupo I (11 pacientes) y grupo II (9 pacientes) similares en sexo, edad y tiempo en diálisis.\nSin diferencias en densidad ósea, PTH, calcio, fósforo y fosfatasa alcalina.\nEl grupo I fue transferido a 2,5 mEq/l y el II continuó con 3 mEq/l.\nSe midieron calcio, fósforo (mg/dl) y fosfatasa alcalina (UI/l) mensualmente.\nPTH (pg/ml) cada tres..."
5,es_ct_ebm_sp_0211-699500012506.ann,concentrado,2556,2567,"... CO3Ca se incrementó: 1.er trimestre: 332 ± 261; 4.º trimestre: 537 ± 260 (expresado como calcio elemento; p < 0,01).\nEn análisis de regresión lineal múltiple, un mayor incremento de PTH y el tipo de concentrado utilizado se comportaron como únicos predictores de pérdida ósea (r = 0,74, p < 0,01).\nEn conclusión, el uso de un concentrado de 2,5 mEq/l de calcio resultó en: 1.º Pérdida de masa ósea. 2.º Aumento..."
6,es_ct_ebm_sp_0211-699500012506.ann,Calcio,2286,2292,"...99 ± 69, p < 0,01) y fosfatasa alcalina (basal: 118,4 ± 47, final: 152 ± 38, p < 0,01), sin cambios en grupo II.\nLa pérdida ósea se relacionó inversamente con incremento de PTH (r = -0,61, p < 0,01).\nCalcio y fósforo no se modificaron.\nEn grupo I la dosis acumulativa de CO3Ca se incrementó: 1.er trimestre: 332 ± 261; 4.º trimestre: 537 ± 260 (expresado como calcio elemento; p < 0,01).\nEn análisis de reg..."
7,es_ct_ebm_sp_0211-699500012506.ann,calcio,2446,2452,"...ncremento de PTH (r = -0,61, p < 0,01).\nCalcio y fósforo no se modificaron.\nEn grupo I la dosis acumulativa de CO3Ca se incrementó: 1.er trimestre: 332 ± 261; 4.º trimestre: 537 ± 260 (expresado como calcio elemento; p < 0,01).\nEn análisis de regresión lineal múltiple, un mayor incremento de PTH y el tipo de concentrado utilizado se comportaron como únicos predictores de pérdida ósea (r = 0,74, p < 0,01..."
8,es_ct_ebm_sp_0211-699500012506.ann,captor,921,927,"...en los pacientes con PTH suprimida.\nEstudiamos 20 pacientes con PTH intacta < 120 pg/ml, 

In [12]:
#table = pd.read_html(html_table)[0] 
# Store the dataframe in Excel file
#table.to_excel("test_table.xlsx")

In [13]:
df_strict

,file,TP,#TP,#gold_entities,#FP,#FN,prec,rec,F1,FP0,...,FN5,FN6,FN7,FN8,FN9,FN10,FN11,FN12,FN13,FN14
0,es_ct_ebm_sp_0211-699500012506.ann,"[(CO3Ca, 910, 915), (calcio, 31, 37), (calcio,...",14,18,12,4,0.54,0.78,0.64,"(CO3Ca, 2357, 2362)",...,None,None,None,None,None,None,None,None,None,None
1,es_ct_ebm_sp_0211-699500012698.ann,"[(quelantes, 386, 395), (calcio, 1480, 1486), ...",25,27,11,2,0.69,0.93,0.79,"(PTH, 1165, 1168)",...,None,None,None,None,None,None,None,None,None,None
2,es_ct_ebm_sp_0211-699502014623.ann,"[(verapamil SR, 890, 902), (losartan, 2352, 23...",15,16,2,1,0.88,0.94,0.91,"(hemoglobina, 1676, 1687)",...,None,None,None,None,None,None,None,None,None,None
3,es_ct_ebm_sp_2010-019912-18.ann,"[(FAMEs, 885, 890), (tocilizumab, 186, 197), (...",4,4,0,0,1.00,1.00,1.00,None,...,None,None,None,None,None,None,None,None,None,None
4,es_ct_ebm_sp_2010-020803-63.ann,"[(masitinib, 223, 232), (corticoides, 328, 339)]",2,2,0,0,1.00,1.00,1.00,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,es_pharmaconer_S1887-85712015000200005-1.ann,[],0,0,0,0,0.00,0.00,0.00,None,...,None,None,None,None,None,None,None,None,None,None
486,es_pharmaconer_S1887-85712017000100004-1.ann,[],0,0,0,0,0.00,0.00,0.00,None,...,None,None,None,None,None,None,None,None,None,None
487,es_pharmaconer_S1888-75462016000400180-1.ann,"[(antiinflamatorios no esteroideos, 902, 934),...",2,2,1,0,0.67,1.00,0.80,"(medicamentos, 154, 166)",...,None,None,None,None,None,None,None,None,None,None
488,es_pharmaconer_S1889-836X2015000200005-1.ann,"[(calcio, 1714, 1720), (magnesio, 1729, 1737),...",7,7,0,0,1.00,1.00,1.00,None,...,None,None,None,None,None,None,None,None,None,None


In [14]:
unique_xps = {"unique_fps_strict": pd.Series(sorted(list(unique_fps_strict))), "unique_fps_lenient": pd.Series(sorted(list(unique_fps_lenient))),
              "unique_fns_strict": pd.Series(sorted(list(unique_fns_strict))), "unique_fns_lenient": pd.Series(sorted(list(unique_fns_lenient)))}
unique_xps_df = pd.DataFrame(unique_xps)
unique_xps_df

,unique_fps_strict,unique_fps_lenient,unique_fns_strict,unique_fns_lenient
0,)A-kappa,)A-kappa,25(OH) vitamina D,25(OH)D
1,-RR,-RR,25(OH)D,25OHD
2,100,100,25OHD,ADR
3,16SrRNA,16SrRNA,ADR,AP
4,25-OH-vitD,25-OH-vitD,ALXN1007,ARN
...,...,...,...,...
361,vitamínicos,NaN,NaN,NaN
362,yodo,NaN,NaN,NaN
363,ácido úrico,NaN,NaN,NaN
364,ácido-base,NaN,NaN,NaN


In [15]:
# df_strict.to_csv(os.path.join(results_dir, "results_ner_strict.csv"))

In [16]:
# df_lenient.to_csv(os.path.join(results_dir, "results_ner_lenient.csv"))

- find error groups
- collect drugs that were not detected (those + drugs from the list could be added in postprocessing step)
- find FNs that were not detected by strict, but by lenient
- check if we can combine words separated by "/"
- get gold annotation and check overlaps with false positives

In [17]:
# write everything to one excel with one sheet per dataframe

def write_to_file(language="all"):
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    writer = pd.ExcelWriter(os.path.join(results_dir, f"error_analysis_ner_{language}.xlsx"), engine='xlsxwriter')

    df_strict.to_excel(writer, sheet_name='strict_overview')

    df_lenient.to_excel(writer, sheet_name='lenient_overview')

    fps_per_file_strict_csv.to_excel(writer, sheet_name='fps_strict')

    fns_per_file_strict_csv.to_excel(writer, sheet_name='fns_strict')

    fps_per_file_lenient_csv.to_excel(writer, sheet_name='fps_lenient')

    fns_per_file_lenient_csv.to_excel(writer, sheet_name='fns_lenient')

    unique_xps_df.to_excel(writer, sheet_name="unique_fps_fns")
    
    writer.save()

In [18]:
write_to_file(language=language)

In [19]:
#unique_fns_strict

In [20]:
# FP = entities that were detected by sys but are not gold
unique_fps_strict

{')A-kappa',
 '-RR',
 '100',
 '16SrRNA',
 '25-OH-vitD',
 'A1AT',
 'ACEITE DE PESCADO',
 'ADN recombinante',
 'AG201',
 'AGE',
 'AN69ST',
 'ANESTÉSICO ÓXIDO NITROSO',
 'ANÁLOGO DE PURINAS CLADRIBINA',
 'ARN VHC',
 'ARP',
 'AST',
 'AclCr',
 'Adipokine',
 'Aglutella',
 'Anticoagulación',
 'Anticuerpos',
 'Antígeno Carbohidrato',
 'Aproten',
 'BIS',
 'Botox',
 'CAV',
 'CLNP023X2202',
 'CO3Ca',
 'CONCENTRADO DE PLASMA AUTÓLOGO RICO',
 'COX',
 'CPK',
 'CQGE031C2201',
 'CRMP',
 'Calcio',
 'Carnoy',
 'Cl. serico',
 'ClCr',
 'Creatina fosfoquinasa',
 'Creatinina',
 'CritiCool',
 'D-dímero',
 'D12',
 'Dako',
 'EDSS',
 'ENEMAS',
 'ETAMSILATO',
 'F alcalina',
 'FACTORES',
 'FAME',
 'FEC',
 'FIBRINA',
 'FOLFOX',
 'FSH',
 'FSH EXÓGENA',
 'GCS',
 'Gram',
 'GranuFoam',
 'HBsAg',
 'HCO3',
 'HMB45',
 'IFN',
 'IL-',
 'IL-6',
 'INTERFERÓN ALFA-2A',
 'IPPVA',
 'ISRS',
 'IgG',
 'Ilieo paralítico',
 'Ineox',
 'Interleukina-2',
 'JNJ-64041575',
 'Ki-67',
 'Ki67',
 'LEOC',
 'Loprofín',
 'Lucentis',
 'MDI basal

In [21]:
unique_fns_lenient

{'25(OH)D',
 '25OHD',
 'ADR',
 'AP',
 'ARN',
 'ARV',
 'AT',
 'Antivipmyn-Tri',
 'BD',
 'BT',
 'C3',
 'CDDP',
 'CFOT',
 'CO3H',
 'COL',
 'CTX',
 'Ca',
 'Cavilon',
 'Cl',
 'Colircusi',
 'Cr',
 'D2',
 'D3',
 'DMSA',
 'DTIC',
 'DaT',
 'Del Nido',
 'Endoperox',
 'Extracto',
 'FDG',
 'FK',
 'FiO2',
 'GS-5734',
 'Glucosa',
 'GranuFoam',
 'HCG',
 'Hormonas tiroideas',
 'IL-6',
 'INMUNOSUPRESOR',
 'K',
 'LCZ696',
 'LDL',
 'LH',
 'Ledermix',
 'MK 0822',
 'MMC',
 'McGhan',
 'Mg',
 'NT-proBNP',
 'Na',
 'Na+',
 'O2',
 'ONI',
 'Ora-Plus',
 'Ora-Sweet',
 'P',
 'PAPP-A',
 'PEG',
 'PRP',
 'PSA',
 'PTH',
 'PTX',
 'Phe',
 'PlGF',
 'Potasio',
 'RBV',
 'RDV',
 'SOF',
 'SSH',
 'SSN',
 'T3',
 'T4',
 'T4 libre',
 'TG',
 'VCR',
 'VP16',
 'Vicryl',
 'Wnt',
 'aFL',
 'albúmina',
 'almagato',
 'alumínica',
 'aminas',
 'amoniaco',
 'anticuerpos',
 'antiinflamatoria',
 'antitrombótica',
 'antiveneno',
 'antiveneno F(ab´)2 polivalente',
 'aprepitant',
 'atole de arroz',
 'bario',
 'bilirrubina',
 'broncodilatador',
 

In [22]:
unique_fps_lenient

{')A-kappa',
 '-RR',
 '100',
 '16SrRNA',
 '25-OH-vitD',
 'A1AT',
 'AG201',
 'AGE',
 'AN69ST',
 'ARP',
 'AST',
 'AclCr',
 'Adipokine',
 'Aglutella',
 'Anticoagulación',
 'Anticuerpos',
 'Aproten',
 'BIS',
 'Botox',
 'CAV',
 'CLNP023X2202',
 'CO3Ca',
 'COX',
 'CPK',
 'CQGE031C2201',
 'CRMP',
 'Calcio',
 'ClCr',
 'Creatina fosfoquinasa',
 'Creatinina',
 'CritiCool',
 'D-dímero',
 'D12',
 'Dako',
 'EDSS',
 'ENEMAS',
 'ETAMSILATO',
 'F alcalina',
 'FAME',
 'FEC',
 'FOLFOX',
 'FSH',
 'GCS',
 'Gram',
 'HBsAg',
 'HMB45',
 'IFN',
 'IL-',
 'IL-6',
 'IPPVA',
 'ISRS',
 'IgG',
 'Ilieo paralítico',
 'Ineox',
 'Interleukina-2',
 'JNJ-64041575',
 'Ki-67',
 'Ki67',
 'LEOC',
 'Loprofín',
 'Lucentis',
 'MDI basal',
 'Maizena',
 'NET',
 'NPT',
 'NYHA',
 'PRP',
 'PTH',
 'Pascal',
 'Persica',
 'PrXPhe',
 'RIA-DIASORIN',
 'RNA',
 'S75-R25',
 'TACHT',
 'TAVI',
 'TCN',
 'TNF',
 'Tirotropina',
 'Tocilizumab',
 'UAA',
 'Vacunas',
 'aceite de girasol',
 'aceites',
 'aceitunas',
 'adipoquinas',
 'agua estéril',
 '

In [23]:
unique_fps_lenient.intersection(unique_fns_lenient)

{'IL-6',
 'PRP',
 'PTH',
 'bilirrubina',
 'calcio',
 'contraste',
 'creatinina',
 'eritropoyetina',
 'fibrinógeno',
 'hemoglobina',
 'medicación',
 'proteína C',
 'proteínas',
 'suero',
 'troponina I'}